## 로지스틱 회귀분석

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import joblib
import numpy as np
import pandas as pd
from scipy import stats
import pingouin as pg

In [ ]:
# 시각화 설정 모듈을 호출합니다.
from GraphicSetting import *

# EDA 시각화 및 통계 분석 관련 모듈을 호출합니다.
import hds_stats as hds

### 실습 데이터셋 준비

In [ ]:
# 인터넷에 공유 중인 텍스트 데이터를 읽고 데이터프레임 df를 생성합니다.
df = pd.read_csv(filepath_or_buffer = 'https://bit.ly/UnivAdmit')

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

In [ ]:
# y절편 역할을 수행할 상수 1을 df의 두 번째 열로 삽입합니다.
df.insert(loc = 1, column = 'const', value = 1)

### 실습 데이터셋 전처리

In [ ]:
# rank를 범주형으로 변환합니다.
df['rank'] = df['rank'].astype(str)

In [ ]:
# 연속형 변수의 기술통계량을 확인합니다.
df.describe()

In [ ]:
# 범주형 변수의 기술통계량을 확인합니다.
df.describe(include = object)

In [ ]:
# rank의 범주별 빈도수를 출력합니다.
df['rank'].value_counts().sort_index()

### 목표변수 시각화

In [ ]:
# 목표변수 범주별 빈도수로 일변량 막대 그래프를 그립니다.
hds.plot.bar_freq(
    data = df, 
    x = 'admit', 
    pal = ['skyblue', 'orange']
)

### 입력변수와 관계 파악: gre

In [ ]:
# admit 범주별 gre의 상자 수염 그림을 그립니다.
hds.plot.box_group(
    data = df, 
    x = 'admit', 
    y = 'gre', 
    pal = ['skyblue', 'orange']
)

### 입력변수와 관계 파악: gpa

In [ ]:
# admit 범주별 gpa의 상자 수염 그림을 그립니다.
hds.plot.box_group(
    data = df, 
    x = 'admit', 
    y = 'gpa', 
    pal = ['skyblue', 'orange']
)

### 입력변수와 관계 파악: rank

In [ ]:
# rank 범주별 admit의 빈도수로 묶음 막대 그래프를 그립니다.
hds.plot.bar_dodge_freq(
    data = df, 
    x = 'rank', 
    group = 'admit', 
    pal = ['skyblue', 'orange']
)

In [ ]:
# rank 범주별 admit의 빈도수로 쌓은 막대 그래프를 그립니다.
hds.plot.bar_stack_freq(
    data = df, 
    x = 'rank', 
    group = 'admit', 
    pal = ['skyblue', 'orange']
)

In [ ]:
# rank 범주별 admit의 상대도수로 쌓은 막대 그래프를 그립니다.
hds.plot.bar_stack_prop(
    data = df, 
    x = 'rank', 
    group = 'admit', 
    pal = ['skyblue', 'orange']
);

### t-검정: gre

In [ ]:
# (정규성 가정 만족) 등분산 검정을 실행합니다.
pg.homoscedasticity(data = df, dv = 'gre', group = 'admit')

In [ ]:
# admit 범주별 gre로 시리즈를 생성합니다.
sp1 = df['gre'][df['admit'].eq('Fail')]
sp2 = df['gre'][df['admit'].eq('Pass')]

In [ ]:
# (정규성 가정 만족) 등분산 가정된 독립표본 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = False)

### t-검정 : gpa

In [ ]:
# (정규성 가정 만족) 등분산 검정을 실행합니다.
pg.homoscedasticity(data = df, dv = 'gpa', group = 'admit')

In [ ]:
# admit 범주별 gpa로 시리즈를 생성합니다.
sp1 = df['gpa'][df['admit'].eq('Fail')]
sp2 = df['gpa'][df['admit'].eq('Pass')]

In [ ]:
# (정규성 가정 만족) 등분산 가정된 독립표본 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = False)

### 교차분석: rank

In [ ]:
# 범주형 입력변수 rank와 목표변수의 교차테이블을 출력합니다.
pd.crosstab(index = df['rank'], 
            columns = df['admit'], 
            margins = True, 
            margins_name = '합계', 
            normalize = 'index')

In [ ]:
# 교차테이블 빈도수로 교차분석(카이제곱 검정)을 실행합니다.
test = pg.chi2_independence(data = df, x = 'rank', y = 'admit')
test[2]

### 더미변수 생성

In [ ]:
# 범주형 입력변수로 더미변수를 생성합니다.
df = pd.get_dummies(data = df, 
                    prefix = ['rank', None], 
                    columns = ['rank', 'admit'], 
                    drop_first = True, 
                    dtype = np.uint8)

In [ ]:
# df의 처음 10행을 출력합니다.
df.head(n = 10)

### 실습 데이터셋 분할

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.model_selection import train_test_split

In [ ]:
# 전체 데이터의 70%를 훈련셋, 30%를 검증셋으로 분할합니다.
train, valid = train_test_split(df, train_size = 0.7, random_state = 0, stratify = df['Pass'])

In [ ]:
# 목표변수명을 변수에 할당합니다.
yvar = 'Pass'

### 입력변수와 목표변수 분리

In [ ]:
# 훈련셋을 입력변수 행렬과 목표변수 벡터로 분리합니다.
trainX = train.drop(columns = [yvar])
trReal = train[yvar].copy()

In [ ]:
# 훈련셋의 목표변수 범주별 상대도수를 확인합니다.
trReal.value_counts(normalize = True)

In [ ]:
# 검증셋을 입력변수 행렬과 목표변수 벡터로 분리합니다.
validX = valid.drop(columns = [yvar])
vaReal = valid[yvar].copy()

In [ ]:
# 검증셋의 목표변수 범주별 상대도수를 확인합니다.
vaReal.value_counts(normalize = True)

### 로지스틱 회귀모형 적합 및 결과 확인

In [ ]:
# 훈련셋으로 로지스틱 회귀모형을 적합합니다.
fit1 = hds.stat.glm(y = trReal, X = trainX)

In [ ]:
# fit1 모형의 적합 결과를 확인합니다.
fit1.summary()

### 로지스틱 회귀모형의 유의성 검정

In [ ]:
# 두 모형의 이탈도 차이를 출력합니다.(검정통계량)
devGap = fit1.null_deviance - fit1.deviance
devGap

In [ ]:
# 두 모형의 자유도 차이를 출력합니다.(카이제곱 분포의 자유도)
dofGap = fit1.df_model.copy()
dofGap

In [ ]:
# 검정통계량과 자유도로 유의확률을 출력합니다.
1 - stats.chi2.cdf(x = devGap, df = dofGap)

### 다중공선성 확인

In [ ]:
# 분산팽창지수를 출력하고 다중공선성 입력변수를 확인합니다.
hds.stat.vif(X = trainX)

### 오즈비 확인

In [ ]:
# 입력변수별 회귀계수의 오즈비를 출력합니다.
np.exp(fit1.params)

### 표준화 회귀계수 확인

In [ ]:
# fit1 모형의 회귀계수를 출력합니다.
fit1.params

In [ ]:
# 표준화 회귀계수를 생성합니다.
beta_z = hds.stat.std_coefs(model = fit1)
beta_z

In [ ]:
# 표준화 회귀계수의 절대값을 오름차순 정렬한 결과를 출력합니다.
beta_z.abs().sort_values()

### 목표변수의 추정확률 생성

In [ ]:
# 훈련셋으로 fit1 모형의 추정확률을 생성하고 실제값과 비교합니다.
trProb = fit1.predict(exog = trainX)
pd.DataFrame(data = {'Real': trReal, 'Prob': trProb})

In [ ]:
# 검증셋으로 fit1 모형의 추정확률을 생성하고 실제값과 비교합니다.
vaProb = fit1.predict(exog = validX)
pd.DataFrame(data = {'Real': vaReal, 'Prob': vaProb})

### 분류모형 성능 평가 : ROC 곡선

In [ ]:
# 훈련셋과 검증셋의 추정확률로 ROC 곡선을 그립니다.
hds.stat.roc_curve(y_true = trReal, y_prob = trProb, color = 'red')
hds.stat.roc_curve(y_true = vaReal, y_prob = vaProb, color = 'blue')

### 목표변수의 추정값 생성

In [ ]:
# 분리 기준점을 0.5로 설정합니다.
cutoff = 0.5

In [ ]:
# 훈련셋의 목표변수 추정값(라벨)을 생성합니다.
trPred1 = np.where(trProb >= cutoff, 1, 0)

In [ ]:
# 검증셋의 목표변수 추정값(라벨)을 생성합니다.
vaPred1 = np.where(vaProb >= cutoff, 1, 0)

### 분류모형 성능 평가 : 혼동행렬 리포트

In [ ]:
# 훈련셋 추정값으로 혼동행렬 리포트를 출력합니다.
hds.stat.clfmetrics(y_true = trReal, y_pred = trPred1)

In [ ]:
# 검증셋 추정값으로 혼동행렬 리포트를 출력합니다.
hds.stat.clfmetrics(y_true = vaReal, y_pred = vaPred1)

### [참고] 혼동행렬 관련 함수

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn import metrics

In [ ]:
# 검증셋 추정값으로 혼동행렬을 출력합니다.
print(metrics.confusion_matrix(y_true = vaReal, y_pred = vaPred1))

In [ ]:
# 검증셋 추정값으로 분류모형 리포트를 출력합니다.
print(metrics.classification_report(y_true = vaReal, y_pred = vaPred1))

In [ ]:
# 검증셋 추정값으로 F1 점수를 출력합니다.
metrics.f1_score(y_true = vaReal, y_pred = vaPred1, pos_label = 1)

### [참고] 목표변수의 범주별 추정확률 분포

In [ ]:
# 목표변수 범주별 추정확률 분포를 비교합니다.
sns.boxplot(x = trReal, y = trProb)
plt.title(label = '범주별 추정확률')
plt.xlabel(xlabel = '실제값')
plt.ylabel(ylabel = '추정확률')
plt.axhline(y = 0.5, color = 'red', lw = 1.5, ls = '--');

### [참고] 최적의 분리 기준점 탐색

In [ ]:
# 분리 기준점마다 민감도와 특이도를 계산하고 두 값의 합계가 최댓값일 때의 
# 분리 기준점을 ROC 곡선 위에 빨간 점으로 표시합니다.
hds.stat.EpiROC(y_true = vaReal, y_prob = vaProb)

### [참고] 최적의 분리 기준점으로 성능지표 확인

In [ ]:
# 최적의 분리 기준점을 설정합니다.
cutoff = 0.27

In [ ]:
# 분리 기준점 변경 후 검증셋의 목표변수 추정값(라벨)을 생성합니다.
vaPred2 = np.where(vaProb >= cutoff, 1, 0)

In [ ]:
# 분리 기준점 변경 후 검증셋 추정값으로 혼동행렬 리포트를 출력합니다.
hds.stat.clfmetrics(y_true = vaReal, y_pred = vaPred2)

In [ ]:
# 분리 기준점 변경 전 검증셋 추정값으로 혼동행렬 리포트와 비교합니다.
hds.stat.clfmetrics(y_true = vaReal, y_pred = vaPred1)

### [참고] 매튜의 상관계수

In [ ]:
# 분리 기준점 변경 전 검증셋 추정값으로 매튜의 상관계수를 출력합니다.
metrics.matthews_corrcoef(y_true = vaReal, y_pred = vaPred1)

In [ ]:
# 분리 기준점 변경 후 검증셋 추정값으로 매튜의 상관계수와 비교합니다.
metrics.matthews_corrcoef(y_true = vaReal, y_pred = vaPred2)

### [참고] 최적의 분리 기준점 탐색 with MCC

In [ ]:
# 분리 기준점 후보를 설정합니다.
cutoffs = np.linspace(start = 0, stop = 1, num = 100 + 1)

In [ ]:
# 매튜의 상관계수를 저장할 빈 리스트를 생성합니다.
mccs = []

In [ ]:
# for 반복문으로 분리 기준점 후보마다 매튜의 상관계수를 계산합니다.
for cutoff in cutoffs:
    vaPred = np.where(vaProb >= cutoff, 1, 0)
    mcc = metrics.matthews_corrcoef(y_true = vaReal, y_pred = vaPred)
    mccs.append(mcc)

In [ ]:
# 최적의 분리 기준점을 반환합니다.
cutoffs[np.argmax(a = mccs)]

## End of Document